In [80]:
from collections import Counter
from hypothesis import given
from hypothesis.strategies import lists, sets, integers, datetimes, sampled_from

SF_18_CHAR_IDS = (
 '00600000002nxMDaay','00600000002nxvTaaq', '00600000002ny2Uaaq', '00600000002okRnaai', '00600000002pObWaau', '00600000002pp1faaa', '00600000002qbSCaay',
 '00600000002qbU2aai', '00600000002qbVJaay', '00600000002sAmSaau', '00600000002tEo5aae', '006000000038DFcaam',
 '00600000003buG6aai', '00600000003bvAcaai', '00Dj0000001ticoeaa', '00600000003bvLqaai', '00600000003c4Dnaai',
 '00600000003DJ9Zaaw', '00T00000005p04beaa', '00Do0000000KjYWea0', '00T0000000bDNIAea4', '00T0000000DsS8Qeav',
 '00T0000000edZNream', '00T0000000gXy1teac', '00T0000000HRq34ead', '00T0000000hs8VMeay', '00T0000000hVn5Heas',
 '00T0000000i2tL2eai', '00T0000000jVXkqeag', '00T0000000kcVEYea2', '00T0000000kG6o6eac', '00T0000000kXnAJea0',
 '00T0000000m8nfTeaq', '00T0000000mA3A2eak', '00T0000000mfyFXeay', '01Jo00000020Wugeae', '01Jo00000020Wuheae',
 '01Jo00000020Wuieae', '01Ji000000KFi7uead', '01JA000000Ix3OGmaz', '01JA000000Ix3OLmaz', '01JA000000IxYXeman',
 '01JA000000IxYXjman', '01JA000000IxYXoman', '01JA000000IxYXtman', '01JA000000JVusAmat', '01JA000000JWi5tmad',
 '01Jb000000CLHGIea5', '01Jb000000CLHGNea5', '00DF00000004zocmaa', '00D60000000IPvgeag', '00D61000000ds5Peaq',
 '00DE0000000J7KQma0', '00D37000000I9cOeas', '00D80000000ZN5Geaw'
)

SF_15_CHAR_IDS = (
    '00600000002nxMD', '00600000002nxvT', '00600000002ny2U', '00600000002okRn', '00600000002pObW',
    '00600000002pp1f', '00600000002qbSC', '00600000002qbU2', '00600000002qbVJ', '00D80000000ZN5G',
    '00600000002sAmS', '00600000002tEo5', '006000000038DFc', '00600000003buG6', '00600000003bvAc',
    '00Dj0000001tico', '00600000003bvLq', '00600000003c4Dn', '00600000003DJ9Z', '00T00000005p04b',
    '00Do0000000KjYW', '00T0000000bDNIA', '00T0000000DsS8Q', '00T0000000edZNr', '00T0000000gXy1t',
    '00T0000000HRq34', '00T0000000hs8VM', '00T0000000hVn5H', '00T0000000i2tL2', '00T0000000jVXkq',
    '00T0000000kcVEY', '00T0000000kG6o6', '00T0000000kXnAJ', '00T0000000m8nfT', '00T0000000mA3A2',
    '00T0000000mfyFX', '01Jo00000020Wug', '01Jo00000020Wuh', '01Jo00000020Wui', '01Ji000000KFi7u',
    '01JA000000Ix3OG', '01JA000000Ix3OL', '01JA000000IxYXe', '01JA000000IxYXj', '01JA000000IxYXo',
    '01JA000000IxYXt', '01JA000000JVusA', '01JA000000JWi5t', '01Jb000000CLHGI', '01Jb000000CLHGN',
    '00DF00000004zoc', '00D60000000IPvg', '00D61000000ds5P', '00DE0000000J7KQ', '00D37000000I9cO'
)

def all_same(*items):
    if not items:
        return True
    return all(x == items[0] for x in items)



# Property-based Testing

#### With examples in Python and JS

### aka Generative Testing


### aka "I'm too lazy to come up with test cases"

# What is generative testing?

Instead of checking a specific output for equality, describe *properties* that should be true of any output.

### Instead of checking for equality...


In [2]:
def test_sort_on_example():
    assert sorted([3, 5, 18, 2, -4]) == [-4, 2, 3, 5, 18]

### Describe a property that should be true.


In [3]:
@given(lists(integers()))
def test_sort_produces_correct_order(a_list):
    sorted_lst = sorted(a_list)
    for ix in range(len(sorted_lst) - 1):
        assert sorted_lst[ix] <= sorted_lst[ix + 1]

# When is generative testing appropriate?

##  When we can come up with properties

- roundtrip computations
- "fixed-point" computations
- compare to a reference implementation
- a clear relationship between input and output

#### How to come up with properties?

## Roundtrip examples -- salesforce ids

Salesforce ids come in two forms:
- case-sensitive 15-char
- case-insensitive 18-char


### Convert from 15 to 18:

1. Split the fifteen characters into three groups of five.
2. For each group of five:
3. Make a length-5 bitstring (eg `01001`) encoding whether each position is capitalized
4. Use a lookup table to map that bitstring to a letter. Tack the letter onto the end.
5. After doing this for each group of five, the last three positions of the id encode the capitalization of the rest of the string.



### Convert from 18 to 15:

Just do the reverse

In [4]:
def sfid_18_to_15(sf_id):
    sf_id = list(sf_id)
    for i in range(3):
        bitmask = 'abcdefghijklmnopqrstuvwxyz012345'.index(sf_id[15 + i])
        for j in range(5):
            if bitmask & (1 << j):
                sf_id[i * 5 + j] = sf_id[i * 5 + j].upper()
            else:
                sf_id[i * 5 + j] = sf_id[i * 5 + j].lower()
    return ''.join(sf_id[:15])


def sfid_15_to_18(sf_id):
    suffix = []
    for i in range(3):
        flags = 0
        for j in range(5):
            c = sf_id[i * 5 + j]
            if 'A' <= c <= 'Z':
                flags += 1 << j
        suffix.append('abcdefghijklmnopqrstuvwxyz012345'[flags])
    return sf_id + ''.join(suffix)


*Can you spot the error?*

## What can we test?



- Passing through both conversions, an 18-char id comes out the same

In [5]:
@given(sampled_from(SF_18_CHAR_IDS))
def test_18_to_15_to_18_roundtrip(sf_18_char):
    assert sf_18_char == sfid_15_to_18(sfid_18_to_15(sf_18_char))

In [6]:
test_18_to_15_to_18_roundtrip()

- Passing through both conversions, a 15-char id comes out the same


In [7]:
@given(sampled_from(SF_15_CHAR_IDS))
def test_15_to_18_to_15_roundtrip(sf_15_char):
    assert sf_15_char == sfid_18_to_15(sfid_15_to_18(sf_15_char))

In [8]:
test_15_to_18_to_15_roundtrip()

*no problems so far, maybe we're good?*

- An 18-char id should be case-insensitive

In [9]:
@given(sampled_from(SF_18_CHAR_IDS))
def test_18_char_is_case_insensitive(sf_18_char):
    assert all_same(
        sfid_18_to_15(sf_18_char),
        sfid_18_to_15(sf_18_char.lower()),
        sfid_18_to_15(sf_18_char.upper()),
        sfid_18_to_15(sf_18_char.swapcase()))


In [10]:
test_18_char_is_case_insensitive()

Falsifying example: test_18_char_is_case_insensitive(sf_18_char='00600000002nxMDaay')


ValueError: substring not found

In [11]:
def sfid_18_to_15(sf_id):
    sf_id = list(sf_id)
    for i in range(3):
        bitmask = 'abcdefghijklmnopqrstuvwxyz012345'.index(sf_id[15 + i].lower()) # <-- added .lower()
        for j in range(5):
            if bitmask & (1 << j):
                sf_id[i * 5 + j] = sf_id[i * 5 + j].upper()
            else:
                sf_id[i * 5 + j] = sf_id[i * 5 + j].lower()
    return ''.join(sf_id[:15])


In [12]:
test_18_char_is_case_insensitive() # now it passes!

#### How to come up with properties?

## Roundtrip examples -- normalize/denormalize

In NREL/openstudio-geometry-editor, we store a geometry object like:

```json
{
  id: 1,
  vertices: [
    {id: 'a', x: 0, y: 16}, {id: 'b', x: 4, y: 16},
    {id: 'c', x: 0, y: 10}, {id: 'd', x: 4, y: 10},
  ],
  edges: [
    { id: 'ab', v1: 'a', v2: 'b' }, { id: 'bd', v1: 'b', v2: 'd' },
    { id: 'cd', v1: 'c', v2: 'd' }, { id: 'ac', v1: 'a', v2: 'c' },
  ],
  faces: [
    {id: 'top', edgeRefs: [
      {edge_id: 'ab', reverse: false}, {edge_id: 'bd', reverse: false},
      {edge_id: 'cd', reverse: true}, {edge_id: 'ac', reverse: true},
    ]},
  ],
}
```

#### How to come up with properties example: normalize/denormalize

Often, a denormalized structure would be more useful for computing on:

```json
{
  id: 1,
  vertices: [
    {id: 'a', x: 0, y: 16}, {id: 'b', x: 4, y: 16},
    {id: 'c', x: 0, y: 10}, {id: 'd', x: 4, y: 10},
  ],
  edges: [
    { id: 'ab', v1: {id: 'a', x: 0, y: 16}, v2: {id: 'b', x: 4, y: 16} },
    { id: 'bd', v1: {id: 'b', x: 4, y: 16}, v2: {id: 'd', x: 4, y: 10} },
    { id: 'cd', v1: {id: 'c', x: 0, y: 10}, v2: {id: 'd', x: 4, y: 10} },
    { id: 'ac', v1: {id: 'a', x: 0, y: 16}, v2: {id: 'c', x: 0, y: 10} },
  ],
  faces: [
    {id: 'top', edgeRefs: [
      { edge: { id: 'ab', v1: {id: 'a', x: 0, y: 16}, v2: {id: 'b', x: 4, y: 16} },
        reverse: false },
      { edge: { id: 'bd', v1: {id: 'b', x: 4, y: 16}, v2: {id: 'd', x: 4, y: 10} },
        reverse: false },
      { edge: { id: 'cd', v1: {id: 'c', x: 0, y: 10}, v2: {id: 'd', x: 4, y: 10} },
        reverse: true },
      { edge: { id: 'ac', v1: {id: 'a', x: 0, y: 16}, v2: {id: 'c', x: 0, y: 10} },
        reverse: true },
    ]},
  ],
}```

#### How to come up with properties example: normalize/denormalize

```javascript
import { gen, check, property } from 'testcheck';

describe('denormalize', () => {
  it('is undone by normalize', () => {
    const result = check(property(
      gen.oneOf(_.values(geometryExamples)),
      (geom) => {
        assertEqual(
          geom,
          normalize(denormalize(geom)),
        );
      },
    ));
    assert(resp.result, 'Property failed to verify!');
  });
});
```

#### Aside: using testcheck.js

`check(property(...))` won't throw an error on it's own. I wrote this helper function:

```javascript
function assertProperty(...args) {
  // if last parameter is not a function, assume it's config for call to check()
  const checkConfig = _.isFunction(args[args.length - 1]) ? {} : args.pop();

  let resp = check(property(...args), checkConfig);
  if (resp.result instanceof Error) {
    resp = resp.shrunk || resp;
    console.error(`${resp.result}`);
    console.error(
      `Property failed to verify on input: ${JSON.stringify(resp.smallest || resp.fail)}`);
    throw resp.result;
  }
  assert(resp.result, `Property failed to verify! ${JSON.stringify(resp)}`);
}
```

#### How to come up with Properties?

## "Fixed-point" computations (aka 'idempotent')

### When applying a function multiple times should give the same result

Examples:

In [13]:
# sorting
@given(lists(integers()))
def test_sort_is_fixed_point(a_list):
    assert sorted(a_list) == sorted(sorted(a_list))

In [14]:
# set operations
@given(sets(integers()), sets(integers()))
def test_set_ops_are_idempotent(set1, set2):
    assert set1 & set2 == (set1 & set2) & set2
    assert set1 | set2 == (set1 | set2) | set2
    assert set1 - set2 == (set1 - set2) - set2

In [15]:
# first_day_of_quarter (a problem I had at my last job)
@given(datetimes())
def test_first_day_of_quarter_is_fixed_point(dt):
    assert first_day_of_quarter(dt) == first_day_of_quarter(first_day_of_quarter(dt))

#### How to come up with Properties?

## Compare to reference function

### why not just *use* the reference function?


- maybe yours is faster


- maybe the reference is in another language?

In [16]:
@given(datetimes(), integers(1, 12))
def test_pg_period_string(date, base_month):
    py_string = period_string(date, 'quarter', base_month)
    db_string = fetchone('SELECT quarter_str(%s, %s)', (base_month, date))[0]
    assert py_string == db_string

- binary search is famously hard to get right, so maybe compare with a linear search?

#### How to come up with Properties?

## Describe a relationship between the inputs and the outputs

In [17]:
@given(lists(integers()), lists(integers()))
def test_list_length_is_preserved_on_addition(a, b):
    assert len(a + b) == len(a) + len(b)

#### How to come up with Properties?

## Start with an example and generalize it

`assertEqual(prime_factors(4), [2, 2])`

Why not test that `prime_factors(2 * any_prime)` is `[that_prime, that_prime]`

```javascript
assertProperty(
  gen.oneOf(first10kPrimes),
  (prime) => assertEqual(primeFactors(2 * prime), [prime, prime]),
);
```

## An Example: merge sort


In [47]:
def merge_sort(lst):
    if len(lst) <= 1:
        return lst
    
    middle = len(lst) // 2
    left = merge_sort(lst[:middle])
    right = merge_sort(lst[middle:])
    
    return merge(left, right)

def merge(left, right):
    if not left or not right:
        # base case: one of the arrays is empty
        return left or right

    result = []
    ix, jx = 0, 0
    while ix < len(left) and jx < len(right):
        if left[ix] <= right[jx]:
            result.append(left[ix])
            ix += 1
        else:
            result.append(right[jx])
            jx += 1
    return result

#### An example: merge sort

## Let's define some tests

In [77]:
@given(lists(integers()))
def test_sort_produces_correct_order(a_list):
    sorted_lst = merge_sort(a_list)
    for ix in range(len(sorted_lst) - 1):
        assert sorted_lst[ix] <= sorted_lst[ix + 1]

@given(lists(integers()))
def test_sort_is_fixed_point(a_list):
    sorted_lst = merge_sort(a_list)
    assert sorted_lst == merge_sort(sorted_lst)

In [62]:
test_sort_produces_correct_order()
test_sort_is_fixed_point()

In [54]:
@given(lists(integers()))
def test_sort_maintains_length(a_list):
    sorted_lst = merge_sort(a_list)
    assert len(sorted_lst) == len(a_list)

In [78]:
@given(lists(integers()))
def test_sorted_list_has_same_elements(a_list):
    sorted_lst = merge_sort(a_list)
    # A Counter is a map from element of the list to the number of times
    # it appears.
    assert Counter(sorted_lst) == Counter(a_list)

In [63]:
@given(lists(integers()))
def test_sort_against_reference(a_list):
    assert merge_sort(a_list) == sorted(a_list)

In [64]:
def printing_exception(func):
    try:
        func()
    except Exception as e:
        print(e)

#### an example: merge sort

Unfortunately, these all fail.

In [79]:
printing_exception(test_sort_maintains_length)
printing_exception(test_sorted_list_has_same_elements)
printing_exception(test_sort_against_reference)

Falsifying example: test_sorted_list_has_same_elements(a_list=[])
name 'Counter' is not defined


In [66]:
merge_sort([0, 0])

[0]

#### an example: merge sort

## Can you spot the error?

In [69]:
def merge_sort(lst):
    if len(lst) <= 1:
        return lst
    
    middle = len(lst) // 2
    left = merge_sort(lst[:middle])
    right = merge_sort(lst[middle:])
    
    return merge(left, right)

def merge(left, right):
    if not left or not right:
        # base case: one of the arrays is empty
        return left or right

    result = []
    ix, jx = 0, 0
    while ix < len(left) and jx < len(right):
        if left[ix] <= right[jx]:
            result.append(left[ix])
            ix += 1
        else:
            result.append(right[jx])
            jx += 1
    return result

We're never copying remaining elements from `left[ix:]` or `right[jx:]`.

#### an example: merge sort

In [73]:
def merge_sort(lst):
    if len(lst) <= 1:
        return lst
    
    middle = len(lst) // 2
    left = merge_sort(lst[:middle])
    right = merge_sort(lst[middle:])
    
    return merge(left, right)

def merge(left, right):
    if not left or not right:
        # base case: one of the arrays is empty
        return left or right

    result = []
    ix, jx = 0, 0
    while ix < len(left) and jx < len(right):
        if left[ix] <= right[jx]:
            result.append(left[ix])
            ix += 1
        else:
            result.append(right[jx])
            jx += 1
    
    # Adding this:
    result.extend(left[ix:])
    result.extend(right[jx:])
    
    
    return result

Now our tests pass...

In [81]:
test_sort_produces_correct_order()
test_sort_is_fixed_point()
test_sort_maintains_length()
test_sorted_list_has_same_elements()
test_sort_against_reference()

## Why do I need a framework

What's wrong with just `import random` or `Math.random()`?